In [2]:
# Пример проведения A/B-теста для uplift-моделирования

# Шаг 1: Определение цели и параметров теста
# Предположим, что цель теста — увеличить количество поездок в сервисе такси после выдачи скидок.
#     Цель: увеличение количества поездок.
#     Метрики: количество поездок, средний чек, общий доход.
#     Период: две недели.

# Шаг 2: Разделение аудитории
# Аудитория делится на две группы случайным образом.

#     Контрольная группа: не получает скидки.
#     Экспериментальная группа: получает скидки.

import pandas as pd
import numpy as np

FILENAME = "discountuplift.csv"

# загрузим данные пользователей
data = pd.read_csv(FILENAME, sep="\t")
data

,recency,history,used_discount,used_bogo,is_referral,treatment,zip_code_Rural,zip_code_Surburban,zip_code_Urban,channel_Multichannel,channel_Phone,channel_Web,proba_CN,proba_CR,proba_TN,proba_TR,uplift_score,target_class
0,3,977.51,1,1,1,0,0,0,1,0,0,1,0.476095,0.159710,0.280660,0.083534,-0.043654,0
1,4,391.10,1,0,0,0,1,0,0,1,0,0,0.443035,0.087289,0.285081,0.184594,0.456858,0
2,7,72.37,1,0,1,1,0,0,1,0,0,1,0.514386,0.015488,0.429174,0.040952,0.115756,2
3,10,67.40,1,0,0,1,0,0,1,0,0,1,0.386800,0.125233,0.415839,0.072128,-0.193534,2
4,2,45.68,1,0,1,1,0,0,1,0,1,0,0.517491,0.022507,0.412372,0.047630,0.123726,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12779,5,104.04,1,0,1,0,0,0,1,0,1,0,0.443421,0.063945,0.450813,0.041821,-0.082280,0
12780,3,558.27,1,0,1,1,0,1,0,0,0,1,0.689539,0.027868,0.224104,0.058488,0.336249,2
12781,11,211.30,1,0,0,0,1,0,0,0,0,1,0.421402,0.076274,0.396279,0.106045,0.115697,0
12782,7,335.62,1,0,1,0,0,1,0,0,0,1,0.531513,0.019835,0.381351,0.067301,0.228063,0


In [3]:
# добавим колонку с группой (контрольная или экспериментальная)
np.random.seed(42)  # для воспроизводимости результатов
data["group"] = np.random.choice(["control", "experiment"], size=len(data))

# сохраним контрольную и экспериментальную группы в отдельные переменные
control_group = data[data["group"] == "control"]
experiment_group = data[data["group"] == "experiment"]
data

,recency,history,used_discount,used_bogo,is_referral,treatment,zip_code_Rural,zip_code_Surburban,zip_code_Urban,channel_Multichannel,channel_Phone,channel_Web,proba_CN,proba_CR,proba_TN,proba_TR,uplift_score,target_class,group
0,3,977.51,1,1,1,0,0,0,1,0,0,1,0.476095,0.159710,0.280660,0.083534,-0.043654,0,control
1,4,391.10,1,0,0,0,1,0,0,1,0,0,0.443035,0.087289,0.285081,0.184594,0.456858,0,experiment
2,7,72.37,1,0,1,1,0,0,1,0,0,1,0.514386,0.015488,0.429174,0.040952,0.115756,2,control
3,10,67.40,1,0,0,1,0,0,1,0,0,1,0.386800,0.125233,0.415839,0.072128,-0.193534,2,control
4,2,45.68,1,0,1,1,0,0,1,0,1,0,0.517491,0.022507,0.412372,0.047630,0.123726,2,control
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12779,5,104.04,1,0,1,0,0,0,1,0,1,0,0.443421,0.063945,0.450813,0.041821,-0.082280,0,experiment
12780,3,558.27,1,0,1,1,0,1,0,0,0,1,0.689539,0.027868,0.224104,0.058488,0.336249,2,experiment
12781,11,211.30,1,0,0,0,1,0,0,0,0,1,0.421402,0.076274,0.396279,0.106045,0.115697,0,experiment
12782,7,335.62,1,0,1,0,0,1,0,0,0,1,0.531513,0.019835,0.381351,0.067301,0.228063,0,control


In [4]:
# Шаг 3: Предоставление версий элемента

#     Контрольная группа: видит стандартные цены без скидок.
#     Экспериментальная группа: видит скидки на поездки.

# добавим колонку с информацией о скидках
data["discount"] = np.where(data["group"] == "experiment", True, False)

# проверим результат
data.head()

,recency,history,used_discount,used_bogo,is_referral,treatment,zip_code_Rural,zip_code_Surburban,zip_code_Urban,channel_Multichannel,channel_Phone,channel_Web,proba_CN,proba_CR,proba_TN,proba_TR,uplift_score,target_class,group,discount
0,3,977.51,1,1,1,0,0,0,1,0,0,1,0.476095,0.159710,0.280660,0.083534,-0.043654,0,control,False
1,4,391.10,1,0,0,0,1,0,0,1,0,0,0.443035,0.087289,0.285081,0.184594,0.456858,0,experiment,True
2,7,72.37,1,0,1,1,0,0,1,0,0,1,0.514386,0.015488,0.429174,0.040952,0.115756,2,control,False
3,10,67.40,1,0,0,1,0,0,1,0,0,1,0.386800,0.125233,0.415839,0.072128,-0.193534,2,control,False
4,2,45.68,1,0,1,1,0,0,1,0,1,0,0.517491,0.022507,0.412372,0.047630,0.123726,2,control,False


In [ ]:
# Шаг 4: Сбор данных
# Собираются данные о количестве поездок, их стоимости, а также другие релевантные метрики в обеих группах.

# создадим DataFrame для хранения результатов теста
results = pd.DataFrame(columns=["user_id", "group", "num_rides", "total_spent"])

# симуляция данных для каждой группы
# для простоты примера мы используем случайные данные
for _, row in data.iterrows():
    num_rides = np.random.poisson(
        lam=5 if row["discount"] else 3
    )  # предполагаем, что скидки увеличивают среднее количество поездок
    total_spent = num_rides * (
        20 if row["discount"] else 25
    )  # допустим, что средняя стоимость поездки отличается

    results = results.append(
        {
            "user_id": row["user_id"],
            "group": row["group"],
            "num_rides": num_rides,
            "total_spent": total_spent,
        },
        ignore_index=True,
    )

# сохраним результаты
results.to_csv("ab_test_results.csv", index=False)

In [ ]:
# Шаг 5: Анализ данных и интерпретация результатов
# После завершения теста анализируются собранные данные:

#     сравниваются результаты контрольной и экспериментальной групп;
#     оценивается, насколько значимо увеличилось количество поездок в экспериментальной группе;
#     применяются статистические тесты для определения значимости различий.

from scipy import stats

# загрузим результаты теста
results = pd.read_csv("ab_test_results.csv")

# группируем данные
grouped_results = results.groupby("group").agg(
    {"num_rides": ["mean", "std", "count"], "total_spent": ["mean", "std", "count"]}
)

print(grouped_results)

# сравниваем средние значения количества поездок
control_rides = results[results["group"] == "control"]["num_rides"]
experiment_rides = results[results["group"] == "experiment"]["num_rides"]

t_stat, p_value = stats.ttest_ind(control_rides, experiment_rides)
print(f"t-statistic: {t_stat}, p-value: {p_value}")

# оцениваем, насколько значимо увеличилось количество поездок в экспериментальной группе
if p_value < 0.05:
    print(
        "Различие в количестве поездок между контрольной и экспериментальной группами статистически значимо."
    )
else:
    print(
        "Различие в количестве поездок между контрольной и экспериментальной группами не является статистически значимым."
    )